In [19]:
%run 'Common.ipynb'

In [87]:
from wndcharm.FeatureSpace import FeatureSpace
f=FeatureSpace.NewFromFileOfFiles( os.path.expanduser('data/medial_7_6_8_delta_3bin.fof'), long=True )
f.ToFitFile('data/medial_7_6_8_delta_3bin.fit')

LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 grp=0 seq=0 fs_col=0>
LOADED  <FeatureVector "T2_map_Resliced_Slice6-l.sig" label="0" n_features=2919 grp=0 seq=0 fs_col=1>
LOADED  <FeatureVector "T2_map_Resliced_Slice8-l.sig" label="0" n_features=2919 grp=0 seq=0 fs_col=2>
LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 grp=1 seq=0 fs_col=0>
LOADED  <FeatureVector "T2_map_Resliced_Slice6-l.sig" label="0" n_features=2919 grp=1 seq=0 fs_col=1>
LOADED  <FeatureVector "T2_map_Resliced_Slice8-l.sig" label="0" n_features=2919 grp=1 seq=0 fs_col=2>
LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 grp=2 seq=0 fs_col=0>
LOADED  <FeatureVector "T2_map_Resliced_Slice6-l.sig" label="0" n_features=2919 grp=2 seq=0 fs_col=1>
LOADED  <FeatureVector "T2_map_Resliced_Slice8-l.sig" label="0" n_features=2919 grp=2 seq=0 fs_col=2>
LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 gr

In [90]:
contig_mat = f.ContiguousDataMatrix()
print (contig_mat)
print (f._contiguous_ground_truth_labels)

mat_vals = [float(x) for x in f._contiguous_ground_truth_labels]
print (mat_vals)

mat_list,class_vals = get_class_mat_list (contig_mat, mat_vals)
print (mat_list)
print (class_vals)


[[  2.46300000e+03   5.44220000e-02   2.34303000e-01 ...,  -4.20041000e+04
    1.33342000e+05   7.16966000e-01]
 [  2.46100000e+03   3.82510000e-02   4.73540000e-02 ...,  -5.46250000e+04
    1.36239000e+05   7.25326000e-01]
 [  2.46300000e+03   2.86680000e-02   2.27273000e-01 ...,  -3.12424000e+04
    1.15157000e+05   7.13205000e-01]
 ..., 
 [  2.40200000e+03   7.63360000e-02   1.62866000e-01 ...,  -6.03631000e+04
    1.30804000e+05   7.21693000e-01]
 [  2.45000000e+03   1.28302000e-01   7.62710000e-02 ...,  -5.37427000e+04
    1.35765000e+05   7.17335000e-01]
 [  2.43000000e+03   1.49250000e-02   4.86660000e-02 ...,  -4.15241000e+04
    1.06458000e+05   7.17794000e-01]]
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '2', '1', '1', '2', '2', '2', '1', '1',

In [ ]:
nfeatures = 200
contig_mat = f.ContiguousDataMatrix()
mat_vals = [float(x) for x in f._contiguous_ground_truth_labels]
mat_list,class_vals = get_class_mat_list (contig_mat, mat_vals)

class_sizes = [m.shape[0] for m in mat_list]
niter = np.product ( class_sizes )
n_classes = len(class_vals)
n_correct = np.asarray( [0]*n_classes)
(train,test) = round_robin_iteration (0,mat_list)
print ('Features          : {}'.format(nfeatures))
print ('Classes           : {}'.format(n_classes))
print ('Class sizes       : {}'.format(class_sizes))
print ('Train class sizes : {}'.format([x.shape[0] for x in train]))
print ('Iterations        : {}'.format(niter))
for iter_idx in range ( niter ):
    # Split
    (train,test) = round_robin_iteration (iter_idx,mat_list)
    (contig_train_mat, contig_train_vals) = list_to_contig_mat (train, class_vals)
    (contig_test_mat, contig_test_vals) = list_to_contig_mat (test, class_vals)

    # Normalize
    (norm_train, norm_test) = normalize (contig_train_mat, contig_test_mat)
    
    # Reduce
    feature_weights = Pearson(norm_train, contig_train_vals)
    (sorted_train, sorted_test) = weigh_sort (norm_train, norm_test, feature_weights)

    # Classify
    preds,pred_vals = WND5(sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)

#    preds = rand_forest_clf (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals, iter_idx)
#    preds = rand_forest_clf (sorted_train[:,:nfeatures], np.asarray([norm_test[1][:,:nfeatures],sorted_test[0][:,:nfeatures]]), contig_train_vals, iter_idx)
    for pred_idx in range (len(preds)):
        if (preds[pred_idx] == contig_test_vals[pred_idx]):
            n_correct[pred_idx] += 1
    cumul_acc = [round ((float(x) / (float(iter_idx)+1.0)),3) for x in n_correct]
    print ('Iteration {}; Predictions: {}; cumulative accuracies: {}'.format(iter_idx, preds, cumul_acc))


    200 features, RFC
    Iteration 1119; Predictions: [ 0.  0.]; cumulative accuracies: [0.73125, 0.7294642857142857]
    200 features, WND5
    Iteration 1119; Predictions: [ 0.  0.]; cumulative accuracies: [0.7366071428571429, 0.7598214285714285]
    100 features, WND5
    Iteration 1119; Predictions: [ 0.  0.]; cumulative accuracies: [0.75, 0.7205357142857143]


In [25]:
contig_mat = f.ContiguousDataMatrix()
print (contig_mat)
print (f._contiguous_ground_truth_labels)

mat_vals = [float(x) for x in f._contiguous_ground_truth_labels]
print (contig_mat)
print (mat_vals)

[[  2.47100000e+03   9.96700000e-03   9.09090000e-02 ...,  -5.68879000e+04
    1.24726000e+05   7.11482000e-01]
 [  2.46700000e+03   8.28920000e-02   4.14750000e-02 ...,  -5.34658000e+04
    1.29016000e+05   7.17171000e-01]
 [  2.30200000e+03   8.79350000e-02   1.46444000e-01 ...,  -6.22379000e+04
    1.28003000e+05   7.28210000e-01]
 ..., 
 [  2.40200000e+03   7.63360000e-02   1.62866000e-01 ...,  -6.03631000e+04
    1.30804000e+05   7.21693000e-01]
 [  2.45000000e+03   1.28302000e-01   7.62710000e-02 ...,  -5.37427000e+04
    1.35765000e+05   7.17335000e-01]
 [  2.43000000e+03   1.49250000e-02   4.86660000e-02 ...,  -4.15241000e+04
    1.06458000e+05   7.17794000e-01]]
['1', '6', '1', '1', '4', '7', '1', '2', '5', '4', '2', '2', '3', '5', '4', '2', '2', '5', '4', '2', '4', '6', '1', '2', '3', '5', '3', '2', '6', '3', '3', '8', '7', '6', '3', '3', '2', '5', '1', '2']
[[  2.47100000e+03   9.96700000e-03   9.09090000e-02 ...,  -5.68879000e+04
    1.24726000e+05   7.11482000e-01]
 [  2.4

In [33]:
contig_mat = f.ContiguousDataMatrix()
mat_vals = [float(x) for x in f._contiguous_ground_truth_labels]

nfeatures = 300
niter = contig_mat.shape[0]
print ('Train class sizes : {}'.format(contig_mat.shape[0]-1))
print ('Features          : {}'.format(nfeatures))
print ('Iterations        : {}'.format(niter))
predictions = []
actual = []
for iter_idx in range ( niter ):
    # Split
    contig_train_mat = np.delete(contig_mat,[iter_idx],axis=0)
    contig_test_mat = np.asarray([contig_mat[iter_idx]])

    contig_train_vals = np.delete (mat_vals, [iter_idx])
    contig_test_vals = np.asarray ([mat_vals[iter_idx]])

    # Normalize
    (norm_train, norm_test) = normalize (contig_train_mat, contig_test_mat)
    
    # Reduce
    feature_weights = Pearson(norm_train, contig_train_vals)
    (sorted_train, sorted_test) = weigh_sort (norm_train, norm_test, feature_weights)

    # Classify
    preds,pred_val = WND5(sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)
    #pred_val = rand_forest_reg (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals, iter_idx)
    #pred_val = lin_reg (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)
    predictions.append (pred_val[0])
    actual.append (mat_vals[iter_idx])
    print ('Iteration {}; Predictions: {}; actual: {}'.format(iter_idx, predictions[-1], actual[-1]))

score, p_value = pearsonr(predictions, actual)
score *= score
print ('R^2: {}, p-value: {}'.format (score, p_value))


Train class sizes : 67
Features          : 300
Iterations        : 68
Iteration 0; Predictions: 0.00113353045954; actual: 0.0
Iteration 1; Predictions: 0.166440586898; actual: 0.0
Iteration 2; Predictions: 0.00142510790085; actual: 0.0
Iteration 3; Predictions: 0.0700558271061; actual: 0.0
Iteration 4; Predictions: 0.00864507282621; actual: 0.0
Iteration 5; Predictions: 0.00864507282621; actual: 0.0
Iteration 6; Predictions: 0.0801706688827; actual: 0.0
Iteration 7; Predictions: 0.116043464772; actual: 0.0
Iteration 8; Predictions: 0.0242067340094; actual: 0.0
Iteration 9; Predictions: 1.33645710659; actual: 0.0
Iteration 10; Predictions: 0.659159930357; actual: 0.0
Iteration 11; Predictions: 1.66344606897; actual: 0.0
Iteration 12; Predictions: 0.970913689947; actual: 0.0
Iteration 13; Predictions: 0.00552222173703; actual: 0.0
Iteration 14; Predictions: 1.97544965227; actual: 0.0
Iteration 15; Predictions: 0.190885000098; actual: 0.0
Iteration 16; Predictions: 0.530753265353; actual:

    200 features RandForReg
    R^2: 0.100336411037, p-value: 0.0464318824205
    20 features lin reg:
    R^2: 0.0491493694884, p-value: 0.169178264743
    20 features RandForReg
    R^2: 0.0595653398403, p-value: 0.129095037849
    100 features RandForReg
    R^2: 0.0395069277481, p-value: 0.218869033287
    100 features RandForReg - binned, 8 bins.
    R^2: 0.0197249101125, p-value: 0.387377038132


